In [26]:
import os
import pandas as pd
import numpy as np

In [27]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [28]:
sheets_desc = {"Data": "مشخصات پرسشنامه",
               "P1":"قسمت یکم: خصوصیات اجتماعی اعضای خانوار",
               "P2": "قسمت دوم: مشخصات محل سکونت",
               "P3S01": "قسمت سوم: بخش ۱ هزینه‌های خوراکی خانوار در ماه گذشته",
               "P3S02": "قسمت سوم: بخش ۲ هزینه‌های نوشیدنی‌های طبقه‌بندی نشده و دخانی های خانوار در ماه گذشته",
               "P3S03": "قسمت سوم: بخش ۳ هزینه‌های پوشاک و کفش خانوار در ماه گذشته",
               "P3S04": "قسمت سوم: بخش ۴ هزینه‌های بخش مسکن، آب، فاضلاب، سوخت و روشنایی خانوار در ماه گذشته",
               "P3S05": "قسمت سوم: بخش ۵ هزینه‌های مبلمان و لوازم خانگی",
               "P3S06": "قسمت سوم: بخش ۶ هزینه‌های بهداشتی و درمانی خانوار در ماه گذشته",
               "P3S07": "قسمت سوم: بخش ۷ هزینه‌های حمل و نقل خانوار در ماه گذشته",
               "P3S08": "قسمت سوم: بخش ۸ هزینه‌های ارتباطات خانوار در ماه گذشته",
               "P3S09": "قسمت سوم: بخش ۹ هزینه‌های خدمات فرهنگی و تفریحات خانوار در ماه گذشته",
               "P3S10": "قسمت سوم: بخش ۱۰ هزینه‌های آموزش و تحصیل",  # empty sheet, included in P3S13
               "P3S11": "قسمت سوم: بخش ۱۱ هزینه‌های غذاهای آماده، هتل و رستوران خانوار در ماه گذشته",
               "P3S12": "قسمت سوم: بخش ۱۲ هزینه‌های کالاها و خدمات متفرقه خانوار در ماه گذشته",
               "P3S13": "قسمت سوم: بخش ۱۳ سایر هزینه‌ها و انتقالات در ۱۲ ماه گذشته",
               "P3S14": "قسمت سوم: بخش ۱۴ سرمایه‌گذاری خانوار در ۱۲ ماه گذشته",
               "P4S01": "قسمت چهارم: بخش ۱ درآمد پولی اعضای شاغل خانوار از مشاغل مزد و حقوق بگیری",
               "P4S02": "قسمت چهارم: بخش ۲ درآمد پولی اعضای شاغل خانوار از مشاغل غیر مزد و حقوق بگیری (آزاد)",
               "P4S03": "قسمت چهارم: بخش ۳ درآمدهای متفرقه خانوار در ۱۲ ماه گذشته",
               "P4S04": "قسمت چهارم: ستون ۹ بخش ۳ شامل وام و یارانه"
               }

In [29]:
sheet_indices = [0,3,4,5,6,7,8,9,10,11,13,14,15,17,18,19,20]

In [30]:
resource_directory = "/content/drive/MyDrive/ai-delta/data.zip (Unzipped Files)"
resource_files : list[str] = []
for f in os.listdir(resource_directory):
    f_path = os.path.join(resource_directory, f)
    if os.path.isfile(f_path):
        resource_files.append(f)
del f, f_path

In [31]:
def empty_string_remover(df):
    for col in df.columns:
    # Checking if string dtype
        if is_string_dtype(df[col]):
            # replace cells with only empty strings with np.nan
            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True)
    return df

In [32]:
dfs = []

for filename in resource_files:

    file_path = os.path.join(resource_directory, filename)
    year_file = pd.ExcelFile(file_path)

    if filename.startswith("U"):
        Urban = True
    else:
        Urban = False

    year = int(filename.strip(".xlsx")[1:])
    if (year==99) or (year ==98):
        year = 1300+year

    for sheet_index in sheet_indices:

        sheet_name = list(sheets_desc.keys())[sheet_index]
        print(f"Sheet name from sheets_desc dict: {sheet_name}.",
            f"Sheet name from file: {year_file.sheet_names[sheet_index]}\n",
            f"Loaded sheet description: {sheets_desc[sheet_name]}")

        if sheet_index==0:
            current_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            current_df = empty_string_remover(current_df)
            # Here we add year and Urban columns to the Data (first sheet) dataframe
            current_df["Urban"] = Urban
            current_df["Year"] = year
            # Drop rows that:
                # TakmilDescA is not being nan, these only exist in year 1401.
                # TakmilDescB or JaygozinDescA or JaygozinDescB or JaygozinDescC is not nan,
                # Jaygozin ==2,
                # However, JaygozinDescC does not appear to have a specific definition.
                # Because all the above mean the end of survey for that family.

            current_df = current_df.loc[current_df.TakmilDescA.isna() |
                                    current_df.TakmilDescB.isna() |
                                    current_df.JaygozinDescA.isna() |
                                    current_df.JaygozinDescB.isna() |
                                    current_df.Jaygozin!=2, :]
            if "JaygozinDescC" in current_df.columns:
                current_df = current_df.loc[current_df.JaygozinDescC.isna(), :]
            year_df = current_df.loc[:, ["Address", "Urban", "Year", "khanevartype", "province", "town"]].copy()
            del current_df


        if sheet_index==3:
            food_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            food_cost_df = (empty_string_remover(food_cost_df ).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "food_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, food_cost_df , how="left", on="Address")
            del food_cost_df

        if sheet_index==4:
            drink_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            drink_cost_df  = (empty_string_remover( drink_cost_df ).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "drink_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, drink_cost_df , how="left", on="Address")
            del drink_cost_df


        if sheet_index==5:
            cloth_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            cloth_cost_df  = (empty_string_remover( cloth_cost_df).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "cloth_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, cloth_cost_df, how="left", on="Address")
            del cloth_cost_df

        if sheet_index==6:
            housing_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            housing_cost_df  = (empty_string_remover(housing_cost_df ).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "housing_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, housing_cost_df  ,how="left", on="Address")
            del housing_cost_df


        if sheet_index==7:
            furniture_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            furniture_cost_df  = (empty_string_remover(furniture_cost_df).groupby("Address")
                                .agg({"value": "sum"})
                                 .rename(columns={"value": "furniture_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, furniture_cost_df , how="left", on="Address")
            del furniture_cost_df

        if sheet_index==8:
            health_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            health_cost_df = (empty_string_remover(health_cost_df).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "health_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, health_cost_df, how="left", on="Address")
            del health_cost_df

        if sheet_index==9:
            transport_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            transport_cost_df = (empty_string_remover(transport_cost_df ).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "transport_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, transport_cost_df , how="left", on="Address")
            del transport_cost_df


        if sheet_index==10:
            communication_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            communication_cost_df= (empty_string_remover(communication_cost_df ).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "communication_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, communication_cost_df , how="left", on="Address")
            del communication_cost_df

        if sheet_index==11:
            entertainment_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            entertainment_cost_df= (empty_string_remover(entertainment_cost_df ).groupby("Address")
                                .agg({"value": "sum"})
                                 .rename(columns={"value": "entertainment_cost"})
                                .reset_index())
            year_df = pd.merge(year_df, entertainment_cost_df, how="left", on="Address")
            del entertainment_cost_df

        if sheet_index==13:
            fastfood_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            fastfood_cost_df= (empty_string_remover(fastfood_cost_df).groupby("Address")
                                .agg({"value": "sum"})
                                 .rename(columns={"value": "fastfood_cost"})
                                .reset_index())
            year_df = pd.merge(year_df,  fastfood_cost_df, how="left", on="Address")
            del  fastfood_cost_df


        if sheet_index==14:
            goods_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            goods_cost_df= (empty_string_remover(   goods_cost_df).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "goods_cost"})
                                .reset_index())
            year_df = pd.merge(year_df,  goods_cost_df, how="left", on="Address")
            del  goods_cost_df


        if sheet_index==15:
            sayer_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            sayer_cost_df= (empty_string_remover( sayer_cost_df).groupby("Address")
                                .agg({"value": "sum"})
                                .rename(columns={"value": "sayer_cost"})
                                .reset_index())
            year_df = pd.merge(year_df,  sayer_cost_df, how="left", on="Address")
            del   sayer_cost_df


          # if sheet_index==16:
          #    sayer_cost_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
          #     sayer_cost_df= (empty_string_remover( sayer_cost_df).groupby("Address")
          #                       .agg({"value": "sum"})
          #                       .reset_index())
          #    year_df = pd.merge(year_df,  sayer_cost_df, how="left", on="Address")
          #    del   sayer_cost_df

        if sheet_index==17:
            income_wage_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            income_wage_df = (empty_string_remover(income_wage_df).groupby("Address")
                                .agg({"netincome_w_y": "sum"})
                                .reset_index())
            year_df = pd.merge(year_df, income_wage_df, how="left", on="Address")
            del income_wage_df

        if sheet_index==18:
            #Calculating azad income
            income_azad_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            income_azad_df = empty_string_remover(income_azad_df).groupby("Address").agg({"income_s_y": "sum"}).reset_index()
            year_df = pd.merge(year_df, income_azad_df, how="left", on="Address")
            del income_azad_df

        if sheet_index==19:
            # Calculating miscellaneous income
            income_misc_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            # income_misc_df = empty_string_remover(income_misc_df).fillna(0)
            income_misc_df=income_misc_df .apply(pd.to_numeric, errors='coerce').astype('Int64')
            income_misc_df = income_misc_df.groupby("Address").agg({"income_pension": "sum",
                                                        "income_rent": "sum",
                                                        "income_interest": "sum",
                                                        "income_aid": "sum",
                                                        "income_resale": "sum",
                                                        "income_transfer": "sum",
                                                        }).sum(axis=1).to_frame(name='misc_income').reset_index()
            year_df = pd.merge(year_df, income_misc_df, how="left", on="Address")
            del income_misc_df

        if sheet_index==20:
            #Calculatin subsidy income
            income_subsidy_df = pd.read_excel(year_file, year_file.sheet_names[sheet_index])
            # income_subsidy_df = empty_string_remover(income_subsidy_df).fillna(0).astype("int64")
            income_subsidy_df=income_subsidy_df .apply(pd.to_numeric, errors='coerce').astype('Int64')
            income_subsidy_df = income_subsidy_df.groupby("Address").agg({"subsidy":"sum"}).reset_index()
            year_df = pd.merge(year_df, income_subsidy_df, how="left", on="Address")
            del income_subsidy_df

    dfs.append(year_df)

Sheet name from sheets_desc dict: Data. Sheet name from file: R1400Data
 Loaded sheet description: مشخصات پرسشنامه
Sheet name from sheets_desc dict: P3S01. Sheet name from file: R1400P3S01
 Loaded sheet description: قسمت سوم: بخش ۱ هزینه‌های خوراکی خانوار در ماه گذشته
Sheet name from sheets_desc dict: P3S02. Sheet name from file: R1400P3S02
 Loaded sheet description: قسمت سوم: بخش ۲ هزینه‌های نوشیدنی‌های طبقه‌بندی نشده و دخانی های خانوار در ماه گذشته
Sheet name from sheets_desc dict: P3S03. Sheet name from file: R1400P3S03
 Loaded sheet description: قسمت سوم: بخش ۳ هزینه‌های پوشاک و کفش خانوار در ماه گذشته
Sheet name from sheets_desc dict: P3S04. Sheet name from file: R1400P3S04
 Loaded sheet description: قسمت سوم: بخش ۴ هزینه‌های بخش مسکن، آب، فاضلاب، سوخت و روشنایی خانوار در ماه گذشته
Sheet name from sheets_desc dict: P3S05. Sheet name from file: R1400P3S05
 Loaded sheet description: قسمت سوم: بخش ۵ هزینه‌های مبلمان و لوازم خانگی
Sheet name from sheets_desc dict: P3S06. Sheet name fr

In [33]:
len(dfs)

8

In [34]:
final_df = pd.concat(dfs, axis=0)

In [35]:
final_df

,Address,Urban,Year,khanevartype,province,town,food_cost,drink_cost,cloth_cost,housing_cost,...,transport_cost,communication_cost,entertainment_cost,fastfood_cost,goods_cost,sayer_cost,netincome_w_y,income_s_y,misc_income,subsidy
0,20001383919,False,1400,1,Markazi,1,14181000.0,NaN,NaN,8110000,...,600000.0,140000.0,NaN,NaN,1150000.0,0480000000000270000000010000000003500000000120...,354000000.0,NaN,158600000.0,23100000
1,20001383923,False,1400,1,Markazi,1,41020000.0,NaN,1500000.0,9000000,...,2800000.0,650000.0,NaN,NaN,2160000.0,0000800000000430000000070000000001500000002500...,NaN,636000000.0,NaN,<NA>
2,20001383925,False,1400,1,Markazi,1,19960000.0,NaN,NaN,8920000,...,2350000.0,600000.0,NaN,NaN,1750000.0,0001800000000070000000022000000000900000000450...,NaN,173000000.0,42320000.0,17640000
3,20001383929,False,1400,1,Markazi,1,24186000.0,NaN,NaN,3560000,...,3000000.0,1000000.0,NaN,NaN,1860000.0,0000228776000050000000006000000000700000,160000000.0,NaN,23560000.0,6720000
4,20001383932,False,1400,1,Markazi,1,48182000.0,NaN,NaN,9260000,...,2500000.0,290000.0,NaN,NaN,1110000.0,0008080000000960000000012500000000100000000800...,NaN,NaN,416360000.0,10920000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19893,13006383810,True,1398,1,Alborz,6,6101000.0,1500000.0,NaN,12530000,...,350000.0,150000.0,NaN,NaN,150000.0,0000300000000950000000012000000000120000000100...,516000000.0,NaN,NaN,16380000
19894,13006383814,True,1398,1,Alborz,6,7200000.0,NaN,2000000.0,12530000,...,290000.0,270000.0,800000.0,NaN,530000.0,0000250000000042000000001000000001200000000015...,216000000.0,NaN,NaN,10920000
19895,13006383815,True,1398,1,Alborz,6,1216666.0,NaN,NaN,5000000,...,NaN,160000.0,NaN,NaN,NaN,NaN,120000000.0,NaN,NaN,<NA>
19896,13006383820,True,1398,1,Alborz,6,6462750.0,750000.0,NaN,10580000,...,3700000.0,540000.0,NaN,NaN,NaN,0000250000000400000000045000000001800000000230...,NaN,360000000.0,NaN,27300000


In [38]:
print(final_df.dtypes)

Address                 int64
Urban                    bool
Year                    int64
khanevartype            int64
province               object
town                    int64
food_cost             float64
drink_cost            float64
cloth_cost            float64
housing_cost            int64
furniture_cost        float64
health_cost           float64
transport_cost        float64
communication_cost    float64
entertainment_cost    float64
fastfood_cost         float64
goods_cost            float64
sayer_cost             object
netincome_w_y         float64
income_s_y             object
misc_income           float64
subsidy                 Int64
dtype: object


In [37]:
final_df_w=final_df.drop(['Urban', 'khanevartype','province','town'], axis=1)

In [40]:
print(final_df_w.dtypes)

Address                 int64
Year                    int64
food_cost             float64
drink_cost            float64
cloth_cost            float64
housing_cost            int64
furniture_cost        float64
health_cost           float64
transport_cost        float64
communication_cost    float64
entertainment_cost    float64
fastfood_cost         float64
goods_cost            float64
sayer_cost             object
netincome_w_y         float64
income_s_y             object
misc_income           float64
subsidy                 Int64
dtype: object


In [47]:
final_df_w.income_s_y  = pd.to_numeric(final_df_w.income_s_y  ,errors='coerce').astype('Int64')
final_df_w['sayer_cost'].astype('int', errors='ignore')

OverflowError: Python int too large to convert to C long

In [48]:
final_df_wn=final_df_w.fillna(0).astype("int64")
final_df_wn.sort_values(by=['Year'], ascending=True)


OverflowError: Python int too large to convert to C long

In [ ]:
final_df_wn

,Address,Year,netincome_w_y,income_s_y,misc_income,subsidy
0,20001383919,1400,354000000,0,158600000,23100000
1,20001383923,1400,0,636000000,0,0
2,20001383925,1400,0,173000000,42320000,17640000
3,20001383929,1400,160000000,0,23560000,6720000
4,20001383932,1400,0,0,416360000,10920000
...,...,...,...,...,...,...
19893,13006383810,1398,516000000,0,0,16380000
19894,13006383814,1398,216000000,0,0,10920000
19895,13006383815,1398,120000000,0,0,0
19896,13006383820,1398,0,360000000,0,27300000


In [ ]:
final_df_wn['sum'] = final_df_wn['netincome_w_y'] +final_df_wn['income_s_y']+final_df_wn['misc_income']+final_df_wn['subsidy']

In [ ]:
final_df_wn

,Address,Year,netincome_w_y,income_s_y,misc_income,subsidy,sum
0,20001383919,1400,354000000,0,158600000,23100000,535700000
1,20001383923,1400,0,636000000,0,0,636000000
2,20001383925,1400,0,173000000,42320000,17640000,232960000
3,20001383929,1400,160000000,0,23560000,6720000,190280000
4,20001383932,1400,0,0,416360000,10920000,427280000
...,...,...,...,...,...,...,...
19893,13006383810,1398,516000000,0,0,16380000,532380000
19894,13006383814,1398,216000000,0,0,10920000,226920000
19895,13006383815,1398,120000000,0,0,0,120000000
19896,13006383820,1398,0,360000000,0,27300000,387300000
